In [1]:
import pandas as pd
import re
import yahooquery as yq
import datetime
import numpy as np

In [100]:
# Leggi il file CSV
df = pd.read_csv('/home/leonardo/Documenti/UNIFI/LAUREA/MATERIALE/hive/tesi28.csv', sep=';')
#pulisci i nome dei df
# sostituisci gli spazi con _ e togli le parentesi
df.columns = [re.sub(r'\(.*\)', '', col) for col in df.columns]
# Rimuovere lo spazio dagli header delle colonne alla fine e tra le parole
df.columns = df.columns.str.rstrip(' ')
df.columns = df.columns.str.rstrip('.')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.replace('/', '_')
df.columns = df.columns.str.replace('.', '')


## scarica i valori di borsa per ogni ticker, per ogni anno

In [101]:
tickers = df['Ticker_symbol']
valori_borsa = []
tickers_non_scaricati=[]

# Scarica i dati storici per ciascun ticker e anno
for ticker in tickers:
    yq_ticker = yq.Ticker(ticker+".mi")
    for year in range(2023, 2013, -1):
        # Specifica la data desiderata come "YYYY-MM-DD" (31 dicembre dell'anno specificato)
        datainizio = f"{year}-01-01"
        datafine = f"{year}-01-10"

        # Converti la data desiderata in un oggetto datetime
        datainizio = datetime.datetime.strptime(datainizio, "%Y-%m-%d").date()
        datafine = datetime.datetime.strptime(datafine, "%Y-%m-%d").date()


        try:
            val = yq_ticker.history(start=datainizio, end=datafine)['close'][-1]
        except Exception as e:
            print(e.__cause__)
            val = "NaN"
            tickers_non_scaricati.append(ticker)

        print(ticker, val)
        valori_borsa.append(val)



MARR 12.300000190734863
MARR 18.81999969482422
MARR 17.799999237060547
MARR 21.200000762939453
MARR 21.520000457763672
MARR 21.739999771118164
MARR 17.649999618530273
MARR 17.969999313354492
MARR 14.75
MARR 12.0
PIA 2.936000108718872
PIA 2.864000082015991
PIA 2.7079999446868896
PIA 2.740000009536743
PIA 1.8949999809265137
PIA 2.503999948501587
PIA 1.621999979019165
PIA 2.009999990463257
PIA 2.4760000705718994
PIA 2.3559999465942383
BRE 11.0600004196167
BRE 12.989999771118164
BRE 10.948678970336914
BRE 11.069999694824219
BRE 10.180000305175781
BRE 13.260000228881836
BRE 12.050000190734863
BRE 8.001999855041504
BRE 5.374000072479248
BRE 3.927999973297119
DAN 22.299999237060547
DAN 26.549999237060547
DAN 15.279999732971191
DAN 16.700000762939453
DAN 16.200000762939453
DAN 21.149999618530273
DAN 20.579999923706055
DAN 15.5
DAN 20.09000015258789
DAN 25.299999237060547
BC 71.5
BC 60.79999923706055
BC 34.2400016784668
BC 35.119998931884766
BC 28.5
BC 28.899999618530273
BC 20.6299991607666
BC 

In [102]:
print(len(valori_borsa))
tickers_non_scaricati = set(tickers_non_scaricati)
print(tickers_non_scaricati)

280
{'GPI', 'ALA', 'ILP', 'FVI', 'ITD', 'LDB', 'STAR7', 'MDC', 'FCM', 'SCK', 'VIM', 'SVS', 'MARP', 'CFL', 'MASI', 'FOS', 'ICF', 'PLT'}


## crea nuovo df
1. prendi info di interesse, moltiplica per ogni anno ogni osservazione 
2. aggiungi gli anni di interesse
3. doppio ciclo for: per ogni riga df originale prendi i primi nove valori e trasponi il risultato, passa ai prossimi nove (secondo ciclo)

#### crea df delle info 

In [ ]:
info = pd.concat([df.iloc[:, 0:5]] * 10)
info.sort_index(inplace=True)
info

#### crea colonna degli anni, aggiungi a info

In [ ]:
anni = [anno for anno in range(2022, 2012, -1)] * 28
info['Anni'] = anni
info

#### df degli indici

In [94]:
# Rimuovi il carattere '\n' e tutti quelli che seguono dalle intestazioni delle colonne
df.columns = df.columns.str.replace(r'\n.*', '', regex=True)
df.columns = df.columns.str.rstrip('_')
nomi_colonne = df.columns.unique()[5:]
# prendi df di interesse, serve a semplificare gli indici
indici = df.iloc[:, 5:]
for i in indici.columns:
    print(i)
# crea nuovo df
new_df = pd.DataFrame()

col = 0
#per ogni set di 10 valori riga di df originario INDICI
for i in range(0, 140, 10):
    
    indice = []
    #per ogni riga AZIENDA
    for j in range(1,len(indici)+1):
        # prendi i nove valori, trasponili e aggiungili al vettore dell'indice considerato
        valori = indici.iloc[j-1:j, i:i+10].values[0]
        valori = valori.T.tolist()
        
        indice= indice + valori
        
    # aggiungi colonna a new_df con insert
    new_df.insert(col, nomi_colonne[col], indice)
    col += 1


for col in new_df.columns:
    new_df[col] = new_df[col].str.replace(',' , '.').astype(float)
    #print(type(X[col]))

new_df

Anni
Valori_Borsa
Indice_corrente
Indice_di_liquidità
Rotaz_cap_investito
Rotaz_cap_cir_lordo
Redditività_del_totale_attivo
Redditività_di_tutto_il_capitale_investito
Redditività_delle_vendite
Redditività_del_capitale_proprio
Indice_di_copertura_delle_immob
Costo_denaro_a_prestito
Oneri_finanz_su_fatt
Grado_di_copertura_degli_interessi_passivi
Debiti_v_banche_su_fatt
Debt_Equity_ratio


ValueError: Length of values (1680) does not match length of index (2800)

#### aggiungi anche valori azionari

In [ ]:
# aggiungi i valori della borsa al df info
info['Valori_Borsa'] = valori_borsa
info

##### esegui il join dei due dataframe, resetta gli indici!!


In [ ]:
#resetta indici
info = info.reset_index(drop=True)
new_df = new_df.reset_index(drop=True)
#esegui il join
dati_puliti = info.join(new_df)
dati_puliti


##### ripulisci i dati, togli le aziende che non hanno valori di brosa per almeno un anno e calcola gli indici per ogni azienda

In [ ]:
# elimina le aziende tramite una maschera
tickers_non_scaricati = list(set(tickers_non_scaricati))
maschera = dati_puliti['Ticker_symbol'].isin(tickers_non_scaricati)
df_pulito = dati_puliti[~maschera]
#df_pulito = df_pulito.drop(columns=["Unnamed: 0"])
df_pulito

### salva il nuovo df 


In [ ]:
df_pulito.to_csv("dati_puliti.csv")

### df con tutte e 28 le aziende

In [ ]:
dati_puliti.to_csv("dati_puliti28.csv")

# CREA IL MODELLO


In [148]:
df = pd.read_csv('/home/leonardo/Documenti/UNIFI/LAUREA/MATERIALE/hive/dati_puliti28.csv', sep=',')
df = df.drop(columns=["Unnamed: 0"])
df['Valori_Borsa'] = np.log(df['Valori_Borsa'])
df

,ID,Ragione_sociale,Ticker_symbol,ISIN_number,Numero_CCIAA,Anni,Valori_Borsa,Indice_corrente,Indice_di_liquidità,Rotaz_cap_investito,...,Redditività_del_totale_attivo,Redditività_di_tutto_il_capitale_investito,Redditività_delle_vendite,Redditività_del_capitale_proprio,Indice_di_copertura_delle_immob,Costo_denaro_a_prestito,Oneri_finanz_su_fatt,Grado_di_copertura_degli_interessi_passivi,Debiti_v_banche_su_fatt,Debt_Equity_ratio
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2022,2.509599,1.58,1.18,1.55,...,3.44,5.34,2.22,7.77,0.45,3.79,0.50,8.11,13.25,0.95
1,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2021,2.934920,1.57,1.20,1.17,...,4.61,7.26,3.83,9.50,0.42,4.31,0.68,8.88,15.76,0.86
2,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2020,2.879198,1.70,1.38,0.98,...,0.09,0.12,0.09,-1.25,0.37,1.60,0.57,5.90,35.37,1.29
3,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2019,3.054001,1.46,1.16,1.46,...,8.84,13.31,5.88,19.42,0.33,2.15,0.40,19.02,18.59,1.04
4,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2018,3.068983,1.67,1.34,1.54,...,9.37,14.36,5.92,20.32,0.20,1.87,0.35,20.30,18.72,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2017,1.324419,3.34,3.31,0.02,...,6.48,7.48,22.97,5.28,0.03,1.42,1.13,25.64,79.55,0.35
276,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2016,1.200567,2.39,2.37,0.02,...,6.51,8.08,24.26,8.55,0.02,2.01,1.26,25.00,62.68,0.27
277,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2015,1.468335,2.55,2.53,0.02,...,7.92,9.94,29.06,13.71,0.02,3.10,1.53,23.10,49.29,0.20
278,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2014,0.816249,2.23,2.21,0.02,...,7.20,9.28,25.64,12.34,0.02,9.10,2.33,13.82,25.64,0.10


In [3]:
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt
import plotly.express as px

print(df.columns[7:])

Index(['Indice_corrente', 'Indice_di_liquidità', 'Rotaz_cap_investito',
       'Rotaz_cap_cir_lordo', 'Redditività_del_totale_attivo',
       'Redditività_di_tutto_il_capitale_investito',
       'Redditività_delle_vendite', 'Redditività_del_capitale_proprio',
       'Indice_di_copertura_delle_immob', 'Costo_denaro_a_prestito',
       'Oneri_finanz_su_fatt', 'Grado_di_copertura_degli_interessi_passivi',
       'Debiti_v_banche_su_fatt', 'Debt_Equity_ratio'],
      dtype='object')


In [116]:
fig = px.scatter(df, y="Valori_Borsa", x="Indice_corrente", color='Ticker_symbol')
fig

In [117]:
fig = px.scatter(df, y="Valori_Borsa", x="Indice_di_liquidità", color='Ticker_symbol')
fig

In [118]:
fig = px.scatter(df, y="Valori_Borsa", x="Rotaz_cap_investito", color='Ticker_symbol')
fig

In [119]:
fig = px.scatter(df, y="Valori_Borsa", x="Rotaz_cap_cir_lordo", color='Ticker_symbol')
fig

In [120]:
fig = px.scatter(df, y="Valori_Borsa", x="Redditività_del_totale_attivo", color='Ticker_symbol')
fig

In [121]:
fig = px.scatter(df, y="Valori_Borsa", x="Redditività_di_tutto_il_capitale_investito", color='Ticker_symbol')
fig

In [122]:
fig = px.scatter(df, y="Valori_Borsa", x="Redditività_delle_vendite", color='Ticker_symbol')
fig

In [123]:
fig = px.scatter(df, y="Valori_Borsa", x="Redditività_del_capitale_proprio", color='Ticker_symbol')
fig

In [124]:
fig = px.scatter(df, y="Valori_Borsa", x="Indice_di_copertura_delle_immob", color='Ticker_symbol')
fig

In [125]:
fig = px.scatter(df, y="Valori_Borsa", x="Costo_denaro_a_prestito", color='Ticker_symbol')
fig

In [126]:
fig = px.scatter(df, y="Valori_Borsa", x="Oneri_finanz_su_fatt", color='Ticker_symbol')
fig

In [127]:
fig = px.scatter(df, y="Valori_Borsa", x="Grado_di_copertura_degli_interessi_passivi", color='Ticker_symbol')
fig

In [128]:
fig = px.scatter(df, y="Valori_Borsa", x="Debiti_v_banche_su_fatt", color='Ticker_symbol')
fig

In [129]:
fig = px.scatter(df, y="Valori_Borsa", x="Debt_Equity_ratio", color='Ticker_symbol')
fig

# prepara il df

In [149]:
# crea le variabili dummy
df_dummies = pd.get_dummies(df["Ticker_symbol"])
df_dummies
# unsci le dummies al df
df_con_dummies = df.join(df_dummies)
df_con_dummies


,ID,Ragione_sociale,Ticker_symbol,ISIN_number,Numero_CCIAA,Anni,Valori_Borsa,Indice_corrente,Indice_di_liquidità,Rotaz_cap_investito,...,MARR,MASI,MDC,PIA,PLT,SCK,SRI,STAR7,SVS,VIM
0,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2022,2.509599,1.58,1.18,1.55,...,True,False,False,False,False,False,False,False,False,False
1,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2021,2.934920,1.57,1.20,1.17,...,True,False,False,False,False,False,False,False,False,False
2,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2020,2.879198,1.70,1.38,0.98,...,True,False,False,False,False,False,False,False,False,False
3,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2019,3.054001,1.46,1.16,1.46,...,True,False,False,False,False,False,False,False,False,False
4,1.0,MARR S.P.A.,MARR,IT0003428445,RN0276618,2018,3.068983,1.67,1.34,1.54,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2017,1.324419,3.34,3.31,0.02,...,False,False,False,False,False,False,False,False,False,False
276,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2016,1.200567,2.39,2.37,0.02,...,False,False,False,False,False,False,False,False,False,False
277,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2015,1.468335,2.55,2.53,0.02,...,False,False,False,False,False,False,False,False,False,False
278,28.0,BASIC NET S.P.A.,BAN,IT0001033700,TO0631153,2014,0.816249,2.23,2.21,0.02,...,False,False,False,False,False,False,False,False,False,False


In [150]:
df_con_dummies.columns

Index(['ID', 'Ragione_sociale', 'Ticker_symbol', 'ISIN_number', 'Numero_CCIAA',
       'Anni', 'Valori_Borsa', 'Indice_corrente', 'Indice_di_liquidità',
       'Rotaz_cap_investito', 'Rotaz_cap_cir_lordo',
       'Redditività_del_totale_attivo',
       'Redditività_di_tutto_il_capitale_investito',
       'Redditività_delle_vendite', 'Redditività_del_capitale_proprio',
       'Indice_di_copertura_delle_immob', 'Costo_denaro_a_prestito',
       'Oneri_finanz_su_fatt', 'Grado_di_copertura_degli_interessi_passivi',
       'Debiti_v_banche_su_fatt', 'Debt_Equity_ratio', 'ALA', 'BAN', 'BC',
       'BRE', 'CFL', 'CLI', 'DAN', 'ELC', 'ENV', 'FCM', 'FOS', 'FVI', 'GPI',
       'ICF', 'ILP', 'ITD', 'LDB', 'MARP', 'MARR', 'MASI', 'MDC', 'PIA', 'PLT',
       'SCK', 'SRI', 'STAR7', 'SVS', 'VIM'],
      dtype='object')

Construct the regression equation. Note that we are leaving out one dummy variable so as to avoid perfect Multicollinearity between the 7 dummy variables. la dummy omessa è BAN

In [161]:
# definisce le var dipendenti e indipendenti
y_var_name = 'Valori_Borsa'
x_var_names =  [
    'Anni',
    'Indice_corrente', 
    'Indice_di_liquidità',
    'Rotaz_cap_investito', 
    'Rotaz_cap_cir_lordo',
    'Redditività_del_totale_attivo',
    'Redditività_di_tutto_il_capitale_investito',
    'Redditività_delle_vendite', 
    'Redditività_del_capitale_proprio',
    'Indice_di_copertura_delle_immob', 
    'Costo_denaro_a_prestito',
    'Oneri_finanz_su_fatt', 
    'Grado_di_copertura_degli_interessi_passivi',
    'Debiti_v_banche_su_fatt', 
    'Debt_Equity_ratio'
    ]

unit_names = [
    'MARR', 
    'PIA', 
    'BRE', 
    'DAN', 
    'BC', 
    'CLI', 
    'ELC', 
    'GPI', 
    'SRI',
    'ALA', 
    'ICF', 
    'ENV', 
    'MASI', 
    'SVS', 
    'STAR7', 
    'SCK', 
    'MARP', 
    'ITD',
    'VIM', 
    'MDC', 
    'ILP', 
    'CFL', 
    'FCM', 
    'FVI', 
    'PLT', 
    'LDB', 
    'FOS',
    'BAN'
    ]

#only_value = [x for x in unit_names if x not in tickers_non_scaricati]
#unit_names = only_value


In [162]:

lsdv_expr = y_var_name + ' ~ '
i = 0
for x_var_name in x_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + x_var_name
    else:
        lsdv_expr = lsdv_expr + x_var_name
    i = i + 1
for dummy_name in unit_names[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name
 
print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Redditività_del_capitale_proprio + Indice_di_copertura_delle_immob + Costo_denaro_a_prestito + Oneri_finanz_su_fatt + Grado_di_copertura_degli_interessi_passivi + Debiti_v_banche_su_fatt + Debt_Equity_ratio + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS


In [163]:
lsdv_model = smf.ols(formula=lsdv_expr, data=df_con_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

                            OLS Regression Results                            
Dep. Variable:           Valori_Borsa   R-squared:                       0.911
Model:                            OLS   Adj. R-squared:                  0.881
Method:                 Least Squares   F-statistic:                     30.78
Date:                Thu, 10 Aug 2023   Prob (F-statistic):           5.15e-48
Time:                        19:23:45   Log-Likelihood:                -24.819
No. Observations:                 166   AIC:                             133.6
Df Residuals:                     124   BIC:                             264.3
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

### previsioni

In [66]:
rmse = {}

rmse

{'Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Grado_di_copertura_degli_interessi_passivi + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS': 4.24345786709551}

In [159]:
import numpy as np

# Ottieni le previsioni per tutte le righe nel dataframe
predictions = lsdv_model_results.predict()

# Calcola l'errore quadratico medio
squared_errors = (predictions - df['Valori_Borsa'].dropna()) ** 2

# Calcola l'RMSE
rmse_val = np.sqrt(squared_errors.mean())

print("RMSE:", rmse_val)

rmse[lsdv_expr] = rmse_val

for k,v in rmse.items():
    print(k, v)


RMSE: 0.28489869254036987
Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Grado_di_copertura_degli_interessi_passivi + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS 4.24345786709551
Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Redditività_del_capitale_proprio + Indice_di_copertura_delle_immob + Costo_denaro_a_prestito + Oneri_finanz_su_fatt + Grado_di_copertura_degli_interessi_passivi + Debiti_v_banche_su_fatt + Debt_Equity_ratio + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM +

In [160]:
rmse_df = pd.DataFrame().from_dict(rmse, orient='index')
rmse_df

,0
Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Grado_di_copertura_degli_interessi_passivi + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,4.243458
Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Redditività_del_capitale_proprio + Indice_di_copertura_delle_immob + Costo_denaro_a_prestito + Oneri_finanz_su_fatt + Grado_di_copertura_degli_interessi_passivi + Debiti_v_banche_su_fatt + Debt_Equity_ratio + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,0.280993
Valori_Borsa ~ Anni + Indice_corrente + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,4.323152
Valori_Borsa ~ Anni + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,4.324254
Valori_Borsa ~ Anni + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + FOS,4.325908
Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Redditività_del_capitale_proprio + Oneri_finanz_su_fatt + Debiti_v_banche_su_fatt + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,4.326915
Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Rotaz_cap_investito + Rotaz_cap_cir_lordo + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Redditività_del_capitale_proprio + Indice_di_copertura_delle_immob + Costo_denaro_a_prestito + Oneri_finanz_su_fatt + Grado_di_copertura_degli_interessi_passivi + Debiti_v_banche_su_fatt + Debt_Equity_ratio + MARR + PIA + BRE + DAN + BC + CLI + ELC + SRI + ENV,5.394634
Valori_Borsa ~ Anni + Indice_corrente + Indice_di_liquidità + Redditività_del_totale_attivo + Redditività_di_tutto_il_capitale_investito + Redditività_delle_vendite + Redditività_del_capitale_proprio + Grado_di_copertura_degli_interessi_passivi + MARR + PIA + BRE + DAN + BC + CLI + ELC + GPI + SRI + ALA + ICF + ENV + MASI + SVS + STAR7 + SCK + MARP + ITD + VIM + MDC + ILP + CFL + FCM + FVI + PLT + LDB + FOS,0.284899
